In [4]:
# %load D:/BUDS_MID/__init__.py
"""
Created on Wed Feb 19 16:21:00 2020

@author: cmchico

usual imports
"""

import os
import pandas as pd
import numpy as np

pd.options.display.max_rows = 100
pd.set_option('display.float_format', '{:.0f}'.format)

from time import time
from datetime import timedelta

import pickle

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

'\nCreated on Wed Feb 19 16:21:00 2020\n\n@author: cmchico\n\nusual imports\n'

In [5]:
path = 'D:/BUDS/ATM Recon/G4S Replenishment Data/'
out = 'D:/BUDS_MID/OJT ATM/ATM Forecasting Data New/'

In [6]:
start = time()
raw= pd.concat([pd.read_excel(path+i).assign(file=i) for i in os.listdir(path)], ignore_index=True)
raw['mon'] = raw.file.str.split(' ').str[3]
print(timedelta(seconds=time()-start))

0:02:10.929554


In [7]:
raw=raw[~raw['Location Id'].isnull()].copy()

In [8]:
raw.mon.value_counts()

Jan    32171
Dec    22703
Nov    20369
Oct    19080
Jul    17412
Jun    17104
Aug    16702
Sep    16552
May    15082
Apr    14589
Feb    13000
Mar     7488
Name: mon, dtype: int64

In [9]:
raw['mon_num2'] = raw.mon.map({'Jan':1,'Feb':2,'Mar':3,'Apr':4,'May':5,'Jun':6,'Jul':7,'Aug':8,'Sep':9,
                              'Oct':10,'Nov':11,'Dec':12})

In [10]:
raw['year_num'] = raw['Job Date'].str.split('/').str[2].astype(int)
raw['day_num'] = raw['Job Date'].str.split('/').str[0].astype(int)
raw['mon_num'] = raw['Job Date'].str.split('/').str[1].astype(int)

In [11]:
raw['Date'] = pd.to_datetime(raw['Job Date'])

select = raw.mon_num == raw.mon_num2
raw.loc[select,'Date'] = pd.to_datetime(raw.loc[select,'year_num'].astype(str)+'/'
                                        +raw.loc[select,'mon_num'].astype(str)+'/'
                                        +raw.loc[select,'day_num'].astype(str))
raw['Job Date'] = raw.Date

In [17]:
df = raw[['Location Id','Job Date','Job Time',
    'Last Load Amount','Current Load Amount','G4S Branch','G$S Branch','% Cash Return/Atm','Replen','Valid Cash Return',
    'Pre-Emptive','Remarks','Site Type','CATEGORY','PREMIUM SITES',
   'Atm Location']].copy()

In [18]:
df.dropna(subset=['Location Id'], inplace=True)
df['G4S Branch'] = df['G4S Branch'].fillna(df['G$S Branch'])
df = df.reset_index(drop=True)
df.drop(columns=['G$S Branch'], inplace=True)

In [19]:
df['_forecasted'] = df.Remarks.fillna("").str.contains('CASTED')
df['_cassettefault'] = df.Remarks.str.contains('XCF') | df.Remarks.str.contains('FAULT')
df['_bpi'] = df['Pre-Emptive'].astype(str).fillna('_').str.contains('BPI')
df['_others'] = (df._forecasted | df._bpi | df._cassettefault) == False
df[['_forecasted','_cassettefault','_bpi','_others']].sum()

_forecasted        73790
_cassettefault    108120
_bpi                7159
_others            30404
dtype: int64

In [22]:
remarks = pd.read_csv(out+'80 Review Replenishment Remarks.csv')

In [39]:
df = df.merge(remarks[['Remarks','FAILURE','FORECAST','BPI','OTHERS']].drop_duplicates(subset='Remarks'))

In [46]:
df['reload_type'] = 'FAILURE'
df.loc[~df.FORECAST.isnull(),'reload_type']="FORECAST"
df.loc[~df.OTHERS.isnull(),'reload_type']="OTHERS"
df.loc[~df.BPI.isnull(),'reload_type']="OTHERS"

In [47]:
df.reload_type.value_counts()

FAILURE     111094
FORECAST     84419
OTHERS       16739
Name: reload_type, dtype: int64

In [48]:
# df[~(df['Pre-Emptive'].astype(str).str.contains('PRE').values)& (df._cassettefault)].Remarks.unique()

In [49]:
# df[df._others].Remarks.unique().tolist()

In [50]:
# df[['Job Date','Replen','Valid Cash Return','Pre-Emptive','Remarks']].drop_duplicates().to_csv(out+'80 Review Replenishment Remarks.csv')

In [51]:
#retain PRE-EMPTIVE DUE TO CF in scenario analysis
# df.groupby(['Replen','Pre-Emptive']).size().reset_index()

In [52]:
#only 1 ATM changed location
# df[['Location Id','Atm Location']].drop_duplicates().groupby(['Location Id']).count()['Atm Location'].value_counts()

In [53]:
fin = df[['Location Id','Job Date','Last Load Amount','Current Load Amount','% Cash Return/Atm','Replen','Pre-Emptive','_forecasted','_cassettefault','_bpi','_others','reload_type']].copy()

In [55]:
fin.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 212252 entries, 0 to 212251
Data columns (total 12 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   Location Id          212252 non-null  float64       
 1   Job Date             212252 non-null  datetime64[ns]
 2   Last Load Amount     212249 non-null  object        
 3   Current Load Amount  212252 non-null  object        
 4   % Cash Return/Atm    212252 non-null  float64       
 5   Replen               212252 non-null  object        
 6   Pre-Emptive          212252 non-null  object        
 7   _forecasted          212252 non-null  bool          
 8   _cassettefault       212252 non-null  bool          
 9   _bpi                 212252 non-null  bool          
 10  _others              212252 non-null  bool          
 11  reload_type          212252 non-null  object        
dtypes: bool(4), datetime64[ns](1), float64(2), object(5)
memory usage: 15.4+

In [56]:
fin['TID'] = fin['Location Id'].astype(int).astype(str)

In [57]:
fin['DATE'] = pd.to_datetime(fin['Job Date'], dayfirst=True)

In [58]:
fin.drop(columns=['Location Id','Job Date']).to_pickle(out+'Actual/80 Replenishment.pkl')